In [19]:
""" A Python Class
A simple Python graph class, demonstrating the essential 
facts and functionalities of graphs.
"""
class Graph:
    def __init__(self, graph_dict=None):
        """ initializes a graph object 
            If no dictionary or None is given, an empty dictionary will be used
        """
        if graph_dict == None:
            graph_dict = {}
        self.__graph_dict = graph_dict

    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()

    def add_vertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.__graph_dict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.__graph_dict:
            self.__graph_dict[vertex] = []

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list; 
            between two vertices can be multiple edges! 
        """
        edge = set(edge) # two distinct vertices would make length 2, otherwise it's a loop
        vertex1 = edge.pop()
        if len(edge) > 0:
            # not a loop
            vertex2 = edge.pop()
        else:
            # a loop
            vertex2 = vertex1
        if vertex1 in self.__graph_dict:
            self.__graph_dict[vertex1].append(vertex2)
        else:
            self.__graph_dict[vertex1] = [vertex2]

    def __generate_edges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.__graph_dict:
            for neighbour in self.__graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges

    def __str__(self):
        res = "vertices: "
        for k in self.__graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.__generate_edges():
            res += str(edge) + " "
        return res

    def find_isolated_vertices(self):
        """ returns a list of isolated vertices. """
        graph = self.__graph_dict
        isolated = []
        for vertex in graph:
            if not graph[vertex]:
                isolated += [vertex]
        return isolated

    def find_path(self, start_vertex, end_vertex, path=[]):
        """ find a path from start_vertex to end_vertex 
            in graph 
            Adjacent vertices: 
                Two vertices are adjacent when they are both incident to a common edge. 
            Simple Path: 
                A path with no repeated vertices is called a simple path. 
                Example: (a, c, e) is a simple path in our graph, 
                as well as (a,c,e,b). (a,c,e,b,c,d) is a path but not
                a simple path, because the node c appears twice. 
        """
        graph = self.__graph_dict
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return path
        if start_vertex not in graph:
            return None
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_path = self.find_path(vertex, 
                                               end_vertex, 
                                               path)
                if extended_path: 
                    return extended_path
        return None
    

    def find_all_paths(self, start_vertex, end_vertex, path=[]):
        """ find all paths from start_vertex to 
            end_vertex in graph """
        graph = self.__graph_dict 
        path = path + [start_vertex]
        if start_vertex == end_vertex:
            return [path]
        if start_vertex not in graph:
            return []
        paths = []
        for vertex in graph[start_vertex]:
            if vertex not in path:
                extended_paths = self.find_all_paths(vertex, 
                                                     end_vertex, 
                                                     path)
                for p in extended_paths: 
                    paths.append(p)
        return paths

    def is_connected(self, 
                     vertices_encountered = None, 
                     start_vertex=None):
        """ determines if the graph is connected 
            A graph is said to be connected if every pair of vertices in the graph 
            is connected. The example graph on the right side is a connected graph. 
            It possible to determine with a simple algorithm whether a graph 
            is connected:
                1. Choose an arbitrary node x of the graph G as the starting point
                2. Determine the set A of all the nodes which can be reached from x.
                If A is equal to the set of nodes of G, the graph is connected; 
                otherwise it is disconnected.
        """
        if vertices_encountered is None:
            vertices_encountered = set()
        gdict = self.__graph_dict        
        vertices = list(gdict.keys())
        if len(vertices) == 0:
            return False
        if not start_vertex:
            # choose a vertex from graph as a starting point
            start_vertex = vertices[0]
        vertices_encountered.add(start_vertex)
        if len(vertices_encountered) != len(vertices):
            for vertex in gdict[start_vertex]:
                if vertex not in vertices_encountered:
                    if self.is_connected(vertices_encountered, vertex):
                        return True
        else:
            return True
        return False

    def vertex_degree(self, vertex):
        """ The degree of a vertex is the number of edges connecting
            it, i.e. the number of adjacent vertices. Loops are counted 
            double, i.e. every occurence of vertex in the list 
            of adjacent vertices. """ 
        adj_vertices =  self.__graph_dict[vertex]
        degree = len(adj_vertices) + adj_vertices.count(vertex)
        return degree

    def degree_sequence(self):
        """ calculates the degree sequence """
        seq = []
        for vertex in self.__graph_dict:
            seq.append(self.vertex_degree(vertex))
        seq.sort(reverse=True)
        return tuple(seq)

    @staticmethod
    def is_degree_sequence(sequence):
        """ Method returns True, if the sequence "sequence" is a 
            degree sequence, i.e. a non-increasing sequence. 
            Otherwise False is returned.
        """
        # check if the sequence sequence is non-increasing:
        return all( x>=y for x, y in zip(sequence, sequence[1:]))

    def delta(self):
        """ the minimum degree of the vertices """
        min = float("inf")
        for vertex in self.__graph_dict:
            vertex_degree = self.vertex_degree(vertex)
            if vertex_degree < min:
                min = vertex_degree
        return min
        
    def Delta(self):
        """ the maximum degree of the vertices """
        max = float("-inf")
        for vertex in self.__graph_dict:
            vertex_degree = self.vertex_degree(vertex)
            if vertex_degree > max:
                max = vertex_degree
        return max

    def density(self):
        """ method to calculate the density of a graph
        The graph density is defined as the ratio of the number of edges of a 
        given graph, and the total number of edges, the graph could have.
        In other words: It measures how close a given graph is to a
        complete graph. 
        The maximal density is 1, if a graph is complete. This is clear,
        because the maximum number of edges in a graph depends on the 
        vertices and can be calculated as: 
        max. number of edges = ½ * |V| * ( |V| - 1 ). 
        On the other hand the minimal density is 0, if the graph has no edges, 
        i.e. it is an isolated graph. 
        For undirected simple graphs, the graph density is defined as:
            D = 2|E|/ |V| * (|V| -1)
        Graph density formula:
            A dense graph is a graph in which the number of edges is close 
            to the maximal number of edges. A graph with only a few edges,
            is called a sparse graph. The definition for those two terms is 
            not very sharp, i.e. there is no least upper bound (supremum) for
            a sparse density and no greatest lower bound (infimum) for defining
            a dense graph. 
        """
        
        g = self.__graph_dict
        V = len(g.keys())
        E = len(self.edges())
        return 2.0 * E / (V *(V - 1))

    def diameter(self):
        """ calculates the diameter of the graph. 
        This means that the diameter is the length of the shortest path
        between the most distanced nodes. To determine the diameter of a 
        graph, first find the shortest path between each pair of vertices.
        The greatest length of any of these paths is the diameter 
        of the graph """
        
        v = self.vertices() 
        pairs = [ (v[i],v[j]) for i in range(len(v)-1) for j in range(i+1, len(v))]
        smallest_paths = []
        for (s,e) in pairs:
            paths = self.find_all_paths(s,e)
            smallest = sorted(paths, key=len)[0]
            smallest_paths.append(smallest)

        smallest_paths.sort(key=len)        

        # longest path is at the end of list, 
        # i.e. diameter corresponds to the length of this path
        diameter = len(smallest_paths[-1])
        return diameter
    
    def shortest_path_between_two_vertices(self, v1, v2):
        """ calculates the shortest path between two vertices in the graph.
        The distance "dist" between two vertices in a graph is the length of the
        shortest path between these vertices. No backtracks, detours, or loops 
        are allowed for the calculation of a distance. 
        """
        paths = self.find_all_paths(v1,v2)
        print("all paths between {v1} and {v2}: {paths}".format(v1=v1, v2=v2, paths=paths))
        smallest = sorted(paths, key=len)[0]
        return smallest
    
    def longest_path_between_two_vertices(self, v1, v2):
        """ calculates the longest path between two vertices in the graph """
        paths = self.find_all_paths(v1,v2)
        print("all paths between {v1} and {v2}: {paths}".format(v1=v1, v2=v2, paths=paths))
        longest = sorted(paths, key=len, reverse=True)[0]
        return longest
    
    def depth_first_search(self, starting_vertex, soughtVertex):
        """ DFS - Depth first search 
         Depth-first search is inherently a recursion:
            1. Start at a vertex.
            2. Pick any unvisited vertex adjacent to the current vertex, 
               and check to see if this is the goal.
            3. If not, recursively apply the depth-first search to that 
               vertex, ignoring any vertices that have already been visited.
            4. Repeat until all adjacent vertices have been visited.
            5. We implement it using STACK.
        """
        graph = self.__graph_dict
        visited_vertices = set()
        stack = [starting_vertex]

        while len(stack) > 0:
            vertex = stack.pop()
            if vertex in visited_vertices:
                continue
            
            visited_vertices.add(vertex)
            
            if vertex == soughtVertex:
                return True

            for v in graph[vertex]:
                if v not in visited_vertices:
                    stack.append(v)
        return False
    
    def breadth_first_search(self, starting_vertex, soughtVertex):
        """ BFS - Breadth first search 
            The breadth-first search operates in the “opposite” way from
            the depth-first search. Intuitively the breadth-first search 
            prefers to visit the neighbors of earlier visited nodes before
            the neighbors of more recently visited ones. We implement it
            using QUEUE.
        """
        from collections import deque
        graph = self.__graph_dict
        visited_vertices = set()
        queue = deque([starting_vertex])

        while len(queue) > 0:
            vertex = queue.popleft()
            if vertex in visited_vertices:
                continue
                
            visited_vertices.add(vertex)
            
            if vertex == soughtVertex:
                return True

            for v in graph[vertex]:
                if v not in visited_vertices:
                    queue.append(v)
            
        return False

if __name__ == "__main__":

    print("===================== g ===================")
    g = { "a" : ["d"],
          "b" : ["c"],
          "c" : ["b", "c", "d", "e"],
          "d" : ["a", "c"],
          "e" : ["c"],
          "f" : []
        }

    for k,v in g.items():
        print(k, ":", v)

    graph = Graph(g)
    print("Vertices of graph:")
    print(graph.vertices())

    print("Edges of graph:")
    print(graph.edges())

    print("Add vertex:")
    graph.add_vertex("z")

    print("Vertices of graph:")
    print(graph.vertices())

    print("Add an edge:")
    graph.add_edge({"a","z"})
    
    print("Print the grapgh using __str__() method:")
    print(graph.__str__())
    
    print("Vertices of graph:")
    print(graph.vertices())
    
    print("Isolated vertices of graph:")
    print(graph.find_isolated_vertices())

    print("Edges of graph:")
    print(graph.edges())

    print('Adding an edge {"x","y"} with new vertices:')
    graph.add_edge({"x","y"})
    print("Vertices of graph:")
    print(graph.vertices())
    print("Edges of graph:")
    print(graph.edges())
    
    print('The path from vertex "a" to vertex "b":')
    path = graph.find_path("a", "b")
    print(path)

    print('The path from vertex "a" to vertex "f":')
    path = graph.find_path("a", "f")
    print(path)

    print('The path from vertex "c" to vertex "c":')
    path = graph.find_path("c", "c")
    print(path)
    
    print("===================== g2 ===================")
    
    g2 = { "a" : ["d", "f"],
      "b" : ["c"],
      "c" : ["b", "c", "d", "e"],
      "d" : ["a", "c"],
      "e" : ["c"],
      "f" : ["d"]
    }

    for k,v in g2.items():
        print(k, ":", v)
    graph2 = Graph(g2)

    print("Vertices of graph:")
    print(graph2.vertices())

    print("Edges of graph:")
    print(graph2.edges())


    print('All paths from vertex "a" to vertex "b":')
    pathAll1 = graph2.find_all_paths("a", "b")
    print(pathAll1)

    print('All paths from vertex "a" to vertex "f":')
    pathAll2 = graph2.find_all_paths("a", "f")
    print(pathAll2)

    print('All paths from vertex "c" to vertex "c":')
    pathAll3 = graph2.find_all_paths("c", "c")
    print(pathAll3)

    print("===================== g3 ===================")
    g3 = { "a" : ["d","f"],
       "b" : ["c","b"],
       "c" : ["b", "c", "d", "e"],
       "d" : ["a", "c"],
       "e" : ["c"],
       "f" : ["a"]
    }

    complete_graph = { 
        "a" : ["b","c"],
        "b" : ["a","c"],
        "c" : ["a","b"]
    }

    isolated_graph = { 
        "a" : [],
        "b" : [],
        "c" : []
    }


    for k,v in g3.items():
        print(k, ":", v)
        
    graph = Graph(g3)
    print("density:", graph.density())

    print("complete_graph:")
    for k,v in complete_graph.items():
        print(k, ":", v)
    graph = Graph(complete_graph)
    print("density:", graph.density())
    
    print("isolated_graph:")
    for k,v in isolated_graph.items():
        print(k, ":", v)
    graph = Graph(isolated_graph)
    print("density:", graph.density())
    
    print("===================== g4 ===================")
    g4 = { "a" : ["d"],
      "b" : ["c"],
      "c" : ["b", "c", "d", "e"],
      "d" : ["a", "c"],
      "e" : ["c"],
      "f" : []
    }
    
    for k,v in g4.items():
        print(k, ":", v)
    graph = Graph(g4)
    print(graph)
    print(graph.is_connected())

    print("===================== g5 ===================")
    g5 = { "a" : ["d","f"],
           "b" : ["c"],
           "c" : ["b", "c", "d", "e"],
           "d" : ["a", "c"],
           "e" : ["c"],
           "f" : ["a"]
    }
    
    for k,v in g5.items():
        print(k, ":", v)
    graph = Graph(g5)
    print(graph)
    print(graph.is_connected())
    
    print("===================== g6 ===================")
    g6 = { "a" : ["d","f"],
           "b" : ["c","b"],
           "c" : ["b", "c", "d", "e"],
           "d" : ["a", "c"],
           "e" : ["c"],
           "f" : ["a"]
    }

    for k,v in g6.items():
        print(k, ":", v)
    graph = Graph(g6)
    print(graph)
    print(graph.is_connected())
    
    print("===================== g7 ===================")
    g7 = { "a" : ["c"],
      "b" : ["c","e","f"],
      "c" : ["a","b","d","e"],
      "d" : ["c"],
      "e" : ["b","c","f"],
      "f" : ["b","e"]
    }

    for k,v in g7.items():
        print(k, ":", v)
    graph = Graph(g7)
    diameter = graph.diameter()
    print(diameter)
    print(graph.shortest_path_between_two_vertices("c", "f"))
    print(graph.longest_path_between_two_vertices("c", "f"))
    
    print("===================== g8 ===================")
    g8 = { "a" : ["d"],
          "b" : ["c"],
          "c" : ["b", "c", "d", "e"],
          "d" : ["a", "c"],
          "e" : ["c"],
          "f" : []
        }

    for k,v in g8.items():
        print(k, ":", v)
    graph = Graph(g8)
    print(graph)

    for node in graph.vertices():
        print(graph.vertex_degree(node))

    print("List of isolated vertices:")
    print(graph.find_isolated_vertices())

    print("""A path from "a" to "e":""")
    print(graph.find_path("a", "e"))

    print("""All pathes from "a" to "e":""")
    print(graph.find_all_paths("a", "e"))

    print("The maximum degree of the graph is:")
    print(graph.Delta())

    print("The minimum degree of the graph is:")
    print(graph.delta())

    print("Edges:")
    print(graph.edges())

    print("Degree Sequence: ")
    ds = graph.degree_sequence()
    print(ds)

    print("Add vertex 'z':")
    graph.add_vertex("z")
    print(graph)

    print("Add edge ('x','y'): ")
    graph.add_edge(('x', 'y'))
    print(graph)

    print("Add edge ('a','d'): ")
    graph.add_edge(('a', 'd'))
    print(graph)
    
    print(graph.depth_first_search('a', 'c')) #true
    print(graph.depth_first_search('a', 'f')) #false
    
    print(graph.breadth_first_search('a', 'c')) #true
    print(graph.breadth_first_search('a', 'f')) #false




===================== g ===================
a : ['d']
b : ['c']
c : ['b', 'c', 'd', 'e']
d : ['a', 'c']
e : ['c']
f : []
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f']
Edges of graph:
[{'d', 'a'}, {'c', 'b'}, {'c'}, {'d', 'c'}, {'e', 'c'}]
Add vertex:
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z']
Add an edge:
Print the grapgh using __str__() method:
vertices: a b c d e f z 
edges: {'d', 'a'} {'c', 'b'} {'c'} {'d', 'c'} {'e', 'c'} {'z', 'a'} 
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z']
Isolated vertices of graph:
['f']
Edges of graph:
[{'d', 'a'}, {'c', 'b'}, {'c'}, {'d', 'c'}, {'e', 'c'}, {'z', 'a'}]
Adding an edge {"x","y"} with new vertices:
Vertices of graph:
['a', 'b', 'c', 'd', 'e', 'f', 'z', 'y']
Edges of graph:
[{'d', 'a'}, {'c', 'b'}, {'c'}, {'d', 'c'}, {'e', 'c'}, {'z', 'a'}, {'y', 'x'}]
The path from vertex "a" to vertex "b":
['a', 'd', 'c', 'b']
The path from vertex "a" to vertex "f":
None
The path from vertex "c" to vertex "c":
['c']
=================

In [ ]:
"""Graph Traversal DFS and BFS - Udacity"""

class Node(object):
    def __init__(self, value):
        self.value = value
        self.edges = []
        self.visited = False

class Edge(object):
    def __init__(self, value, node_from, node_to):
        self.value = value
        self.node_from = node_from
        self.node_to = node_to

# You only need to change code with docs strings that have TODO.
# Specifically: Graph.dfs_helper and Graph.bfs
# New methods have been added to associate node numbers with names
# Specifically: Graph.set_node_names
# and the methods ending in "_names" which will print names instead
# of node numbers

class Graph(object):
    def __init__(self, nodes=None, edges=None):
        self.nodes = nodes or []
        self.edges = edges or []
        self.node_names = []
        self._node_map = {}

    def set_node_names(self, names):
        """The Nth name in names should correspond to node number N.
        Node numbers are 0 based (starting at 0).
        """
        self.node_names = list(names)

    def insert_node(self, new_node_val):
        "Insert a new node with value new_node_val"
        new_node = Node(new_node_val)
        self.nodes.append(new_node)
        self._node_map[new_node_val] = new_node
        return new_node

    def insert_edge(self, new_edge_val, node_from_val, node_to_val):
        "Insert a new edge, creating new nodes if necessary"
        nodes = {node_from_val: None, node_to_val: None}
        for node in self.nodes:
            if node.value in nodes:
                nodes[node.value] = node
                if all(nodes.values()):
                    break
        for node_val in nodes:
            nodes[node_val] = nodes[node_val] or self.insert_node(node_val)
        node_from = nodes[node_from_val]
        node_to = nodes[node_to_val]
        new_edge = Edge(new_edge_val, node_from, node_to)
        node_from.edges.append(new_edge)
        node_to.edges.append(new_edge)
        self.edges.append(new_edge)

    def get_edge_list(self):
        """Return a list of triples that looks like this:
        (Edge Value, From Node, To Node)"""
        return [(e.value, e.node_from.value, e.node_to.value)
                for e in  self.edges]

    def get_edge_list_names(self):
        """Return a list of triples that looks like this:
        (Edge Value, From Node Name, To Node Name)"""
        return [(edge.value,
                 self.node_names[edge.node_from.value],
                 self.node_names[edge.node_to.value])
                for edge in self.edges]

    def get_adjacency_list(self):
        """Return a list of lists.
        The indecies of the outer list represent "from" nodes.
        Each section in the list will store a list
        of tuples that looks like this:
        (To Node, Edge Value)"""
        max_index = self.find_max_index()
        adjacency_list = [[] for _ in range(max_index)]
        for edg in self.edges:
            from_value, to_value = edg.node_from.value, edg.node_to.value
            adjacency_list[from_value].append((to_value, edg.value))
        return [a or None for a in adjacency_list] # replace []'s with None

    def get_adjacency_list_names(self):
        """Each section in the list will store a list
        of tuples that looks like this:
        (To Node Name, Edge Value).
        Node names should come from the names set
        with set_node_names."""
        adjacency_list = self.get_adjacency_list()
        def convert_to_names(pair, graph=self):
            node_number, value = pair
            return (graph.node_names[node_number], value)
        def map_conversion(adjacency_list_for_node):
            if adjacency_list_for_node is None:
                return None
            return map(convert_to_names, adjacency_list_for_node)
        return [map_conversion(adjacency_list_for_node)
                for adjacency_list_for_node in adjacency_list]

    def get_adjacency_matrix(self):
        """Return a matrix, or 2D list.
        Row numbers represent from nodes,
        column numbers represent to nodes.
        Store the edge values in each spot,
        and a 0 if no edge exists."""
        max_index = self.find_max_index()
        adjacency_matrix = [[0] * (max_index) for _ in range(max_index)]
        for edg in self.edges:
            from_index, to_index = edg.node_from.value, edg.node_to.value
            adjacency_matrix[from_index][to_index] = edg.value
        return adjacency_matrix

    def find_max_index(self):
        """Return the highest found node number
        Or the length of the node names if set with set_node_names()."""
        if len(self.node_names) > 0:
            return len(self.node_names)
        max_index = -1
        if len(self.nodes):
            for node in self.nodes:
                if node.value > max_index:
                    max_index = node.value
        return max_index

    def find_node(self, node_number):
        "Return the node with value node_number or None"
        return self._node_map.get(node_number)
    
    def _clear_visited(self):
        for node in self.nodes:
            node.visited = False

    def dfs_helper(self, start_node):
        """TODO: Write the helper function for a recursive implementation
        of Depth First Search iterating through a node's edges. The
        output should be a list of numbers corresponding to the
        values of the traversed nodes.
        ARGUMENTS: start_node is the starting Node
        MODIFIES: the value of the visited property of nodes in self.nodes 
        RETURN: a list of the traversed node values (integers).
        """
        ret_list = [start_node.value]
        # Your code here
        start_node.visited = True
        edges_out = [e for e in start_node.edges
                     if e.node_to.value != start_node.value]
        for edge in edges_out:
            if not edge.node_to.visited:
                ret_list.extend(self.dfs_helper(edge.node_to))
        return ret_list

    def dfs(self, start_node_num):
        """Outputs a list of numbers corresponding to the traversed nodes
        in a Depth First Search.
        ARGUMENTS: start_node_num is the starting node number (integer)
        MODIFIES: the value of the visited property of nodes in self.nodes
        RETURN: a list of the node values (integers)."""
        self._clear_visited()
        start_node = self.find_node(start_node_num)
        return self.dfs_helper(start_node)

    def dfs_names(self, start_node_num):
        """Return the results of dfs with numbers converted to names."""
        return [self.node_names[num] for num in self.dfs(start_node_num)]

    def bfs(self, start_node_num):
        """TODO: Create an iterative implementation of Breadth First Search
        iterating through a node's edges. The output should be a list of
        numbers corresponding to the traversed nodes.
        ARGUMENTS: start_node_num is the node number (integer)
        MODIFIES: the value of the visited property of nodes in self.nodes
        RETURN: a list of the node values (integers)."""
        node = self.find_node(start_node_num)
        self._clear_visited()
        ret_list = [node.value]
        queue = [node]
        node.visited = True
        def enqueue(n, q=queue):
            n.visited = True
            q.append(n)
        def unvisited_outgoing_edge(n, e):
            return ((e.node_from.value == n.value) and
                    (not e.node_to.visited))
        while queue:
            node = queue.pop(0)
            ret_list.append(node.value)
            for e in node.edges:
                if unvisited_outgoing_edge(node, e):
                    enqueue(e.node_to)
        return ret_list

    def bfs_names(self, start_node_num):
        """Return the results of bfs with numbers converted to names."""
        return [self.node_names[num] for num in self.bfs(start_node_num)]

graph = Graph()

# You do not need to change anything below this line.
# You only need to implement Graph.dfs_helper and Graph.bfs

graph.set_node_names(('Mountain View',   # 0
                      'San Francisco',   # 1
                      'London',          # 2
                      'Shanghai',        # 3
                      'Berlin',          # 4
                      'Sao Paolo',       # 5
                      'Bangalore'))      # 6 

graph.insert_edge(51, 0, 1)     # MV <-> SF
graph.insert_edge(51, 1, 0)     # SF <-> MV
graph.insert_edge(9950, 0, 3)   # MV <-> Shanghai
graph.insert_edge(9950, 3, 0)   # Shanghai <-> MV
graph.insert_edge(10375, 0, 5)  # MV <-> Sao Paolo
graph.insert_edge(10375, 5, 0)  # Sao Paolo <-> MV
graph.insert_edge(9900, 1, 3)   # SF <-> Shanghai
graph.insert_edge(9900, 3, 1)   # Shanghai <-> SF
graph.insert_edge(9130, 1, 4)   # SF <-> Berlin
graph.insert_edge(9130, 4, 1)   # Berlin <-> SF
graph.insert_edge(9217, 2, 3)   # London <-> Shanghai
graph.insert_edge(9217, 3, 2)   # Shanghai <-> London
graph.insert_edge(932, 2, 4)    # London <-> Berlin
graph.insert_edge(932, 4, 2)    # Berlin <-> London
graph.insert_edge(9471, 2, 5)   # London <-> Sao Paolo
graph.insert_edge(9471, 5, 2)   # Sao Paolo <-> London
# (6) 'Bangalore' is intentionally disconnected (no edges)
# for this problem and should produce None in the
# Adjacency List, etc.

import pprint
pp = pprint.PrettyPrinter(indent=2)

print("Edge List")
pp.pprint(graph.get_edge_list_names())

print("\nAdjacency List")
adj_list = []
for i in graph.get_adjacency_list_names():
    if i == None:
        adj_list.append(i)
    else:
        adj_list.append(list(i))
pp.pprint(adj_list)

print("\nAdjacency Matrix")
pp.pprint(graph.get_adjacency_matrix())

print("\nDepth First Search")
pp.pprint(graph.dfs_names(2))

# Should print:
# Depth First Search
# ['London', 'Shanghai', 'Mountain View', 'San Francisco', 'Berlin', 'Sao Paolo']

print("\nBreadth First Search")
pp.pprint(graph.bfs_names(2))
# test error reporting
# pp.pprint(['Sao Paolo', 'Mountain View', 'San Francisco', 'London', 'Shanghai', 'Berlin'])

# Should print:
# Breadth First Search
# ['London', 'Shanghai', 'Berlin', 'Sao Paolo', 'Mountain View', 'San Francisco']

In [ ]:
"""
============= FINDING SHORTEST PATHS: Tips ====================

1. If it's an unweighted graph, then the shortest
path is basically BFS (the path with least number of edges 
leading to the destination vertex).

2. If it's a weighted graph, then the shortest
path is basically the path that has the least total weight
of all the edges leading to the destination vertex.

3. If it's a weighted undirected graph, then one solution is Dijkstra's Algorithm (below).

"""

In [ ]:
"""
============ Dijkstra's Algorithm -- Shortest Path ===========
If it's a weighted undirected graph, then we use Min Priority Queue: 
    # Time: O(|V|^2) or O(|E| + |V|log|V|)
    # Space: O(|V|)

"""




In [10]:
def find_max(matrix):
    longest_paths_coordinate = None
    longest_paths_length = 0
    rows = len(matrix)
    cols = len(matrix[0])
    i = 0
    j = 0
    
    for i in range(0, rows):
        for j in range(0, cols):
            if matrix[i][j] == 0:
                curr_coordinate_paths = 1
                for col in range(j+1, cols):
                    if matrix[i][col] == 1: 
                        break
                    if matrix[i][col] == 0: 
                        curr_coordinate_paths += 1
                
                for col in range(j-1, -1, -1):
                    if matrix[i][col] == 1: 
                        break
                    if matrix[i][col] == 0: 
                        curr_coordinate_paths += 1
                    
                for row in range(i+1, rows):
                    if matrix[row][j] == 1: 
                        break
                    if matrix[row][j] == 0: 
                        curr_coordinate_paths += 1
                            
                for row in range(i-1, -1, -1):
                    if matrix[row][j] == 1: 
                        break
                    if matrix[row][j] == 0: 
                        curr_coordinate_paths += 1
                
                print("matrix[row][col] coord = ({},{}), Number of paths: {}".format(i,j,curr_coordinate_paths))
                if curr_coordinate_paths > longest_paths_length:
                    longest_paths_length = curr_coordinate_paths
                    longest_paths_coordinate = (i,j)
            else: 
                print("matrix[row][col] coord = ({},{}), Number of paths: {}".format(i,j,'WALL'))
                
    return {longest_paths_coordinate, longest_paths_length}

matrix = [[0,0,1,1,0],
          [1,0,1,0,0],
          [0,0,1,1,0],
          [0,1,0,1,0]]
find_max(matrix)

matrix[row][col] coord = (0,0), Number of paths: 2
matrix[row][col] coord = (0,1), Number of paths: 4
matrix[row][col] coord = (0,2), Number of paths: WALL
matrix[row][col] coord = (0,3), Number of paths: WALL
matrix[row][col] coord = (0,4), Number of paths: 4
matrix[row][col] coord = (1,0), Number of paths: WALL
matrix[row][col] coord = (1,1), Number of paths: 3
matrix[row][col] coord = (1,2), Number of paths: WALL
matrix[row][col] coord = (1,3), Number of paths: 2
matrix[row][col] coord = (1,4), Number of paths: 5
matrix[row][col] coord = (2,0), Number of paths: 3
matrix[row][col] coord = (2,1), Number of paths: 4
matrix[row][col] coord = (2,2), Number of paths: WALL
matrix[row][col] coord = (2,3), Number of paths: WALL
matrix[row][col] coord = (2,4), Number of paths: 4
matrix[row][col] coord = (3,0), Number of paths: 2
matrix[row][col] coord = (3,1), Number of paths: WALL
matrix[row][col] coord = (3,2), Number of paths: 1
matrix[row][col] coord = (3,3), Number of paths: WALL
matrix[

{(1, 4), 5}